### Setup e Variáveis

In [ ]:
from notebooks.core.bemol_lakestorage import BemolLakeStorage
from core.bemol_controller import BemolController
from core.bemol_logger import BemolLogger
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

In [ ]:
# Instanciando classe BemolLogger
logger = BemolLogger("silver_products_sales")

# Configurando Spark com Delta Lake
spark = (
  SparkSession.builder
  .appName("TransformacaoSilverProdutos")
  .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
  .getOrCreate()
)

# Instanciando classe de leitura/escrita de dados BemolLakeStorage
lakehouse = BemolLakeStorage(spark, logger)

In [ ]:
# Definindo paths de origem e destino
origin_path_carts = "../data/bronze/carts/"
origin_path_products = "../data/bronze/products/"
destination_path = "../data/silver/products_sales/"

# Path para salvar os dados de monitoramento
destination_path_monitor = "../data/monitoring/"

### Leitura

In [ ]:
# Lendo dados da camada bronze através do método read_bronze
df_carts = lakehouse.read_bronze(origin_path_carts)
df_products = lakehouse.read_bronze(origin_path_products)

### Tranformações

In [ ]:
# Remove duplicatas
df_products = df_products.dropDuplicates(["id"])

# Agrega a quantidade total vendida por produto
df_sales = df_carts.groupBy("product_id").agg(
    sum("product_quantity").alias("total_quantity_sold")
)

In [ ]:
# Enriquecer o dataframe de produtos com a quantidade total vendida
df_products_silver = df_products.join(df_sales, df_products.id == df_sales.product_id, "left").drop(df_sales.product_id)

# Seleciona as colunas relevantes
df_products_silver = df_products_silver.select("id", "product_title", "category", "rating_count", "rating", "price", "total_quantity_sold")

# Substitui valores nulos na coluna total_quantity_sold por 0
df_products_silver = df_products_silver.fillna(0, subset=["total_quantity_sold"])

# Calcula a receita total por produto
df_products_silver = df_products_silver.withColumn("total_revenue", col("price") * col("total_quantity_sold"))

### Escrita

In [ ]:
# Adiciona coluna de controle através do método control_field
df_products_silver = BemolController.control_field(df_products_silver, "silver")

In [ ]:
# Grava os dados na camada Silver no formato Delta, utilizando o modo overwrite como padrão através do método write_silver
lakehouse.write_silver(df_products_silver, destination_path, table_name="silver_products_sales")

In [ ]:
# Grava os dados de monitoramento no formato Delta, utilizando o modo overwrite como padrão através do método export_delta
lakehouse.monitor.export_delta(spark, destination_path_monitor)